# Mini Project 1

In [1]:
print('Hello MP1!')


Hello MP1!


### Imported libraries used for the project
1. jupiter
2. scikit-learn
3. gensim
4. nltk
5. numpy
6. pandas
7. matplotlib

`conda install jupiter scikit-learn gensim nltk numpy pandas matplotlib`

## 1. Dataset Preparation & Analysis (5pts)

1.2. Load the dataset. You can use `gzip.open` and `json.load` to do that.

In [2]:
import gzip
import json

dataset = gzip.open('goemotions.json.gz')
dataset_json = json.load(dataset)

# Close the gz dataset once your finished loading the data as a json object
dataset.close()


1.3. (5pts) Extract the posts and the 2 sets of labels (emotion and sentiment), then plot the distribution
of the posts in each category and save the graphic (a histogram or pie chart) in pdf. Do this for both
the emotion and the sentiment categories. You can use `matplotlib.pyplot` and `savefig` to do this.
This pre-analysis of the dataset will allow you to determine if the classes are balanced, and which
metric is more appropriate to use to evaluate the performance of your classifiers.

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

numpy_dataset = np.array(dataset_json)

# Get column only for emotion and sentiment
emotion_dataset_col = numpy_dataset[:, 1]
sentiment_dataset_col = numpy_dataset[:, 2]

# Count the number of times each value appears
emotion_count = Counter(emotion_dataset_col)
sentiment_count = Counter(sentiment_dataset_col)

# Save the data values as a histogram
# plt.hist(emotion_count.values())
# plt.savefig('emotions_graph')
# plt.hist(emotion_count)
# plt.savefig('emotions_graph2')

plt.pie(emotion_count.values(), None, emotion_count.keys())
plt.savefig('emotions_pie_chart')

plt.close()

# plt.hist(sentiment_count.values())
# plt.savefig('sentiment_graph')
# plt.hist(sentiment_count)
# plt.savefig('sentiment_graph2')
plt.pie(sentiment_count.values(), None, sentiment_count.keys())
plt.savefig('sentiment_pie_chart')

plt.close()


In [4]:
print(emotion_count, emotion_count.keys())

Counter({'neutral': 55298, 'approval': 11259, 'admiration': 10531, 'annoyance': 8342, 'disapproval': 7686, 'gratitude': 7075, 'amusement': 6130, 'curiosity': 5885, 'anger': 5202, 'love': 4957, 'confusion': 4938, 'realization': 4714, 'disappointment': 4706, 'optimism': 4519, 'joy': 4329, 'sadness': 3827, 'caring': 3523, 'surprise': 3472, 'excitement': 3020, 'disgust': 2914, 'desire': 2147, 'fear': 1778, 'remorse': 1510, 'embarrassment': 1433, 'nervousness': 796, 'relief': 788, 'pride': 690, 'grief': 351}) dict_keys(['sadness', 'neutral', 'love', 'gratitude', 'disapproval', 'amusement', 'disappointment', 'realization', 'admiration', 'annoyance', 'confusion', 'optimism', 'excitement', 'caring', 'remorse', 'joy', 'approval', 'embarrassment', 'surprise', 'curiosity', 'anger', 'grief', 'disgust', 'pride', 'desire', 'relief', 'fear', 'nervousness'])


## 2. Words as Features (35pts)

2.1. □ (5pts) Process the dataset using `feature_extraction.text.CountVectorizer` to extract tokens/words
and their frequencies. Display the number of tokens (the size of the vocabulary) in the dataset.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Phrases are in the first column of the dataset
phrases = numpy_dataset[:, 0]

# Process the dataset
vectorizer_emotions = CountVectorizer()

# X value is the processed_dataset
X_emotions = vectorizer_emotions.fit_transform(phrases)

print("Number of features (tokens in the vocabulary) =",
      len(vectorizer_emotions.get_feature_names_out()))


Number of features (tokens in the vocabulary) = 30449


In [6]:
emotions = numpy_dataset[:, 1]

emotions_and_phrases = phrases.copy()

for count, i in enumerate(phrases):
    emotions_and_phrases[count] = i + " " + emotions[count]

vectorizer_sentiments = CountVectorizer()
X_sentiments = vectorizer_sentiments.fit_transform(emotions_and_phrases)

print("Number of features (tokens in the vocabulary) including emotions =",
      len(vectorizer_sentiments.get_feature_names_out()))


Number of features (tokens in the vocabulary) including emotions = 30450


2.2. □ (2pts) Split the dataset into 80% for training and 20% for testing. For this, you can use `train_test_split`.

In [7]:
# Split the dataset
from sklearn.model_selection import train_test_split

# Split the dataset
training_dataset, testing_dataset = train_test_split(
    numpy_dataset, train_size=0.8, test_size=0.2)

# Split the feature vector of emotions
training_X_emotions, testing_X_emotions = train_test_split(
    X_emotions, train_size=0.8, test_size=0.2)

# Split the feature vector of sentiments
training_X_sentiments, testing_X_sentiments = train_test_split(
    X_sentiments, train_size=0.8, test_size=0.2)

# Print the size of both datasets
print("Size of training set =", training_dataset.shape[0])
print("Size of testing set =", testing_dataset.shape[0])


Size of training set = 137456
Size of testing set = 34364


2.3. Train and test the following classifiers, for both the emotion and the sentiment classification, using
word frequency as features.

* 2.3.1. □ (3pts) **Base-MNB**: a Multinomial Naive Bayes Classifier `(naive_bayes.MultinomialNB.html)`
with the default parameters.

In [8]:
from sklearn.naive_bayes import MultinomialNB

# Create the object classifiers for emotions
emotions_classifier_mb = MultinomialNB()

# Fit the model with training_X as X and columns of training_dataset as y
emotions_classifier_mb.fit(X=training_X_emotions,
                           y=training_dataset[:, 1])

# Make predictions with testing_X as X
emotion_prediction_mb = emotions_classifier_mb.predict(X=testing_X_emotions)
print(emotion_prediction_mb)


['neutral' 'neutral' 'neutral' ... 'neutral' 'optimism' 'neutral']


In [9]:
# Part 2.4 for Multinomial classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Append Emotions results
performance_file = open("performance", "w")
performance_file.write(
    "-----Emotions classification (Multinomial Naive Bayes)-----\n")

performance_file.write(
    f"Emotions hyperparameters = {emotions_classifier_mb.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 1], y_pred=emotion_prediction_mb)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 1], y_pred=emotion_prediction_mb, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")

performance_file.close()


In [10]:
# Create the object classifiers for sentiments
sentiment_classifier_mb = MultinomialNB()

# Fit the model with training_X as X and columns of training_dataset as y
sentiment_classifier_mb.fit(X=training_X_sentiments,
                            y=training_dataset[:, 2])

# Make predictions with testing_X as X
sentiment_prediction_mb = sentiment_classifier_mb.predict(
    X=testing_X_sentiments)
print(sentiment_prediction_mb)


['negative' 'neutral' 'positive' ... 'neutral' 'neutral' 'positive']


In [11]:
# Append Sentiments results
performance_file = open("performance", "a")
performance_file.write(
    "-----Sentiments classification (Multinomial Naive Bayes)-----\n")

performance_file.write(
    f"Sentiments hyperparameters = {sentiment_classifier_mb.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 2], y_pred=sentiment_prediction_mb)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 2], y_pred=sentiment_prediction_mb, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")

performance_file.close()


* 2.3.2. □ (3pts) **Base-DT:** a Decision Tree `(tree.DecisionTreeClassifier)` with the default parameters.

In [12]:
from sklearn.tree import DecisionTreeClassifier

# Create the object classifiers for emotions
emotions_classifier_dt = DecisionTreeClassifier()

# Fit the model with training_X as X and columns of training_dataset as y
emotions_classifier_dt.fit(X=training_X_emotions,
                           y=training_dataset[:, 1])

# Make predictions with testing_X as X
emotion_prediction_dt = emotions_classifier_dt.predict(X=testing_X_emotions)
print(emotion_prediction_dt)


In [ ]:
# Part 2.4 for DecisionTree classification


# Append Emotions results
performance_file = open("performance", "a")
performance_file.write("-----Emotions classification (Decision Tree)-----\n")

performance_file.write(
    f"Emotions hyperparameters = {emotions_classifier_dt.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 1], y_pred=emotion_prediction_dt)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 1], y_pred=emotion_prediction_dt, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")
performance_file.close()


In [ ]:
# Create the object classifiers for sentiments
sentiment_classifier_dt = DecisionTreeClassifier()

# Fit the model with training_X as X and columns of training_dataset as y
sentiment_classifier_dt.fit(X=training_X_sentiments,
                            y=training_dataset[:, 2])

# Make predictions with testing_X as X
sentiment_prediction_dt = sentiment_classifier_dt.predict(
    X=testing_X_sentiments)
print(sentiment_prediction_dt)


['positive' 'ambiguous' 'neutral' ... 'neutral' 'neutral' 'positive']


In [ ]:

# Append Sentiments results
performance_file = open("performance", "a")
performance_file.write(
    "-----Sentiments classification (Decision Tree)-----\n")

performance_file.write(
    f"Sentiments hyperparameters = {sentiment_classifier_dt.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 2], y_pred=sentiment_prediction_dt)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 2], y_pred=sentiment_prediction_dt, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")

performance_file.close()


* 2.3.3. □ (3pts) **Base-MLP:** a Multi-Layered Perceptron `(neural network.MLPClassifier)` with the
default parameters.

In [ ]:
# import MLPClassifier
from sklearn.neural_network import MLPClassifier

# Create the object classifiers for emotions
emotions_classifier_mlp = MLPClassifier(verbose=True,max_iter=3)

# Fit the model with training_X as X and columns of training_dataset as y
emotions_classifier_mlp.fit(X=training_X_emotions, y=training_dataset[:, 1])

# Make predictions with testing_X as X
emotion_prediction_mlp = emotions_classifier_mlp.predict(X=testing_X_emotions)
print(emotion_prediction_mlp)

Iteration 1, loss = 2.77578435
Iteration 2, loss = 2.62992284
Iteration 3, loss = 2.47876997
['neutral' 'neutral' 'neutral' ... 'neutral' 'neutral' 'neutral']


c:\users\zosan\pycharmprojects\pythonproject\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [ ]:
# Part 2.4 for MLP classification


# Append Emotions results
performance_file = open("performance", "a")
performance_file.write("-----Emotions classification (multi layer perceptron)-----\n")

performance_file.write(
    f"Emotions hyperparameters = {emotions_classifier_mlp.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 1], y_pred=emotion_prediction_mlp)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 1], y_pred=emotion_prediction_mlp, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")
performance_file.close()


In [ ]:
# Create the object classifiers for sentiments
sentiment_classifier_mlp = MLPClassifier(verbose=True,max_iter=3)

# Fit the model with training_X as X and columns of training_dataset as y
sentiment_classifier_mlp.fit(X=training_X_sentiments, y=training_dataset[:, 2])

# Make predictions with testing_X as X
sentiment_prediction_mlp = sentiment_classifier_mlp.predict(X=testing_X_sentiments)

print(sentiment_prediction_mlp)

Iteration 1, loss = 1.31506083
Iteration 2, loss = 1.28044700
Iteration 3, loss = 1.21299265
['positive' 'positive' 'neutral' ... 'positive' 'positive' 'neutral']


c:\users\zosan\pycharmprojects\pythonproject\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [ ]:
# Append Sentiments results
performance_file = open("performance", "a")
performance_file.write(
    "-----Sentiments classification (Multi layer perceptron)-----\n")

performance_file.write(
    f"Sentiments hyperparameters = {sentiment_classifier_mlp.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 2], y_pred=sentiment_prediction_mlp)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 2], y_pred=sentiment_prediction_mlp, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")

performance_file.close()

* 2.3.4. □ (3pts) **Top-MNB:** a better performing Multinomial Naive Bayes Classifier found using `GridSearchCV`.
The gridsearch will allow you to find the best combination of hyper-parameters, as determined
by the evaluation function that you have determined in step 1.3. The only hyper-parameter that
you will experiment with is `alphafloat` with values 0.5, 0 and 2 other values of your choice.

* 2.3.5. □ (3pts) **Top-DT:** a better performing Decision Tree found using `GridSearchCV.` The hyperparameters
that you will experiment with are:
  * `criterion:` gini or entropy
  * `max depth:` 2 different values of your choice
  * `min samples split:` 3 different values of your choice

In [ ]:
from sklearn.model_selection import GridSearchCV
# apply hyper parameters
param_grid = {'criterion': ['gini', 'entropy'],
              'max_depth': [2, 4], 'min_samples_split': [2, 5, 10]}

# create objects classifier with grid search
grid_search_emotion_classifier = GridSearchCV(
    emotions_classifier_dt, param_grid)

# Fit the model with training_X as X and columns of training_dataset as y
grid_search_emotion_classifier.fit(
    X=training_X_emotions, y=training_dataset[:, 1])

# Make predictions with testing_X as X
grid_search_emotion_predict = grid_search_emotion_classifier.predict(
    X=testing_X_emotions)
print(grid_search_emotion_predict)


['neutral' 'neutral' 'neutral' ... 'neutral' 'neutral' 'neutral']


In [ ]:
# Part 2.4 for gridsearch Dt classification


# Append Emotions results
performance_file = open("performance", "a")
performance_file.write("-----Emotions classification (Grid Search Decision Tree)-----\n")

performance_file.write(
    f"Emotions hyperparameters = {grid_search_emotion_classifier.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 1], y_pred=grid_search_emotion_predict)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 1], y_pred=grid_search_emotion_predict, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")
performance_file.close()

In [ ]:
# create objects classifier with grid search
grid_search_sentiment_classifier = GridSearchCV(sentiment_classifier_dt, param_grid)

# Fit the model with training_X as X and columns of training_dataset as y
grid_search_sentiment_classifier.fit(X=training_X_sentiments, y=training_dataset[:, 2])

# Make predictions with testing_X as X
grid_search_sentiment_predict = grid_search_sentiment_classifier.predict(X=testing_X_sentiments)
print(grid_search_sentiment_predict)

['positive' 'positive' 'positive' ... 'positive' 'positive' 'positive']


In [ ]:
# Append Sentiments results
performance_file = open("performance", "a")
performance_file.write(
    "-----Sentiments classification (Grid Search Decision Tree)-----\n")

performance_file.write(
    f"Sentiments hyperparameters = {grid_search_sentiment_classifier.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 2], y_pred=grid_search_sentiment_predict)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 2], y_pred=grid_search_sentiment_predict, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")

performance_file.close()

* 2.3.6. □ (3pts) **Top-MLP:** a better performing Multi-Layered Perceptron found using GridSearchCV.
The hyper-parameters that you will experiment with are:
    * `activation:` sigmoid, tanh, relu and identity
    * 2 network architectures of your choice: for eg, 2 hidden layers with 30+50 nodes and 3 hidden
layers with 10 + 10 + 10
    * `solver:` Adam and stochastic gradient descent

2.4. □ (5pts) For each of the 6 classifiers above and each of the classification tasks (emotion or sentiment),
produce and save the following information in a file called `performance`:
* a string clearly describing the model (e.g. the model name + hyper-parameter values) and the
classification task (emotion or sentiment)
* the confusion matrix – use `metrics.confusion_matrix`
* the precision, recall, and F1-measure for each class, and the accuracy, macro-

2.5. □ (7.5pts) **Do your own exploration:** Do only one of the following, depending on your own interest:
* Use tf-idf instead of word frequencies and redo all substeps of 2.3 above – you can use `TfidfTransformer`
for this. Display the results of this experiment.
* Remove stop words and redo all substeps of 2.3 above – you can use the parameter of `CountVectorizer`
for this. Display the results of this experiment.
* Play with `train_test_split` in order have different splits of 80% training, 20% test sets and
different sizes of training sets and redo all substeps of 2.3 above. Show and explain how the
performance of your models vary depending on the training/test sets are used.

## 3. Embeddings as Features (20pts)

3.1. □ (0pts) Use `gensim.downloader.load` to load the `word2vec-google-news-300` pretrained embedding model.

In [ ]:
# Incase its the first time running
# import nltk

# nltk.download('punkt')


In [ ]:
import gensim.downloader
google_model = gensim.downloader.load("word2vec-google-news-300", return_path=True)


3.2. □ (2pts) Use the `tokenizer` from `nltk` to extract words from the Reddit posts. Display the number
of tokens in the training set.

In [ ]:
import nltk.tokenize


phrase_token_vector = [nltk.tokenize.word_tokenize(i) for i in phrases]
emotions_and_phrases_token_vector = [nltk.tokenize.word_tokenize(i) for i in emotions_and_phrases]


In [ ]:
# Get the token count in phrase token list
token_count = 0
for i in phrase_token_vector:
    token_count += len(i)

print("Number of Tokens in phrases=", token_count)


# Get the token count in phrase and emotion token list
token_count = 0
for i in emotions_and_phrases_token_vector:
    token_count += len(i)

print("Number of Tokens in phrases=", token_count)


3.3. □ (5pts) Compute the embedding of a Reddit post as the <u>average</u> of the embeddings of its words. If
a word has no embedding in Word2Vec, skip it.

In [ ]:
from statistics import mean
from gensim.models import Word2Vec, KeyedVectors


mean_embedding_list_emotions = []
for i in phrase_token_vector:
    mean_embedding_list_emotions.append(google_model.get_mean_vector(i))

mean_embedding_list_sentiments = []
for i in emotions_and_phrases_token_vector:
    mean_embedding_list_sentiments.append(google_model.get_mean_vector(i))


# Test to get the average embedding for a phrase with index i
i = 0
print(
    f"The mean embedding for phrase: '{phrase_token_vector[i]}'\n = \n{mean_embedding_list_emotions[i]}")

# Test to get the average embedding for a pohrase and emotion with index i
i = 0
print(
    f"The mean embedding for phrase and emotion: '{emotions_and_phrases_token_vector[i]}'\n = \n{mean_embedding_list_sentiments[i]}")


3.4. □ (3pts) Compute and display the overall hit rates of the training and test sets (i.e. the % of words
in the Reddit posts for which an embedding is found in Word2Vec).


In [ ]:
# Split the tokens (to get the hit rate)
training_phrase_token, testing_phrase_token = train_test_split(
    phrase_token_vector, train_size=0.8, test_size=0.2)

training_emotions_and_phrases_token, testing_emotions_and_phrases_token = train_test_split(
    emotions_and_phrases_token_vector, train_size=0.8, test_size=0.2)


# Split the mean embedding list for both eomtions and sentiments (the X values)
training_X_emotions_emb, testing_X_emotions_emb = train_test_split(
    mean_embedding_list_emotions, train_size=0.8, test_size=0.2)

training_X_sentiments_emb, testing_X_sentiments_emb = train_test_split(
    mean_embedding_list_sentiments, train_size=0.8, test_size=0.2)


In [ ]:
# Hit Rate for phrases training set
num_of_hits = 0
total = 0

for i in training_phrase_token:
    for word in i:
        try:
            # pdb.set_trace()
            google_model.key_to_index[word]
            num_of_hits += 1
        except Exception:
            pass
        total += 1

print(f"Hit Rate in training set (phrases) = {num_of_hits/total*100}%")


# Hit Rate for phrases testing set
num_of_hits = 0
total = 0

for i in testing_phrase_token:
    for word in i:
        try:
            # pdb.set_trace()
            google_model.key_to_index[word]
            num_of_hits += 1
        except Exception:
            pass
        total += 1

print(f"Hit Rate in testing set (phrases) = {num_of_hits/total*100}%")

# Hit Rate for phrases and emotions training set
num_of_hits = 0
total = 0

for i in training_emotions_and_phrases_token:
    for word in i:
        try:
            # pdb.set_trace()
            google_model.key_to_index[word]
            num_of_hits += 1
        except Exception:
            pass
        total += 1

print(f"Hit Rate in training set (phrases and emotions) = {num_of_hits/total*100}%")


# Hit Rate for phrases and emotions testing set
num_of_hits = 0
total = 0

for i in testing_emotions_and_phrases_token:
    for word in i:
        try:
            # pdb.set_trace()
            google_model.key_to_index[word]
            num_of_hits += 1
        except Exception:
            pass
        total += 1

print(f"Hit Rate in testing set (phrases and emotions) = {num_of_hits/total*100}%")


3.5. □ (3pts) **Train a Base-MLP:** a Multi-Layered Perceptron (`neural_network.MLPClassifier`) with
the default parameters.

In [ ]:
from sklearn.neural_network import MLPClassifier


# For emotions classification
emotions_classifier_mlp_embedding = MLPClassifier()

# Fit the model with training_X as X and columns of training_dataset as y
emotions_classifier_mlp_embedding.fit(X=training_X_emotions_emb,
                                      y=training_dataset[:, 1])

# Make predictions with testing_X as X
emotion_prediction_mlp_embedding = emotions_classifier_mlp_embedding.predict(
    X=testing_X_emotions_emb)
print(emotion_prediction_mlp_embedding)


In [ ]:
# Append Emotions results
performance_file = open("performance", "a")
performance_file.write(
    "-----Emotions classification (google model) (Multi Layered Perception with word Embeddings)-----\n")

performance_file.write(
    f"Emotions hyperparameters = {emotions_classifier_mlp_embedding.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 1], y_pred=emotion_prediction_mlp_embedding)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 1], y_pred=emotion_prediction_mlp_embedding, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")

performance_file.close()

In [ ]:
# For sentiments classification
sentiments_classifier_mlp_embedding = MLPClassifier()

# Fit the model with training_X as X and columns of training_dataset as y
sentiments_classifier_mlp_embedding.fit(X=training_X_sentiments_emb,
                                      y=training_dataset[:, 2])

# Make predictions with testing_X as X
sentiments_prediction_mlp_embedding = sentiments_classifier_mlp_embedding.predict(
    X=testing_X_sentiments_emb)
print(sentiments_prediction_mlp_embedding)

In [ ]:
# Append Sentiments results
performance_file = open("performance", "a")
performance_file.write(
    "-----Sentiments classification (google model) (Multi Layered Perception with word Embeddings)-----\n")

performance_file.write(
    f"Sentiments hyperparameters = {sentiments_classifier_mlp_embedding.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 2], y_pred=sentiments_prediction_mlp_embedding)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 2], y_pred=sentiments_prediction_mlp_embedding, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")

performance_file.close()


3.6. □ (3pts) **Train a Top-MLP:** a better performing Multi-Layered Perceptron found with whatever
hyperparameters you want.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

# hyperparameter used in gridsearch
hyperparam = {'activation': ['sigmoid', 'tanh', 'relu', 'identity'],
                'hidden_layer_sizes': [2, 3],
                'solver': ['Adam', 'sgd']}

# For emotions classification
emotions_classifier_top_mlp_gridsearch_emb = GridSearchCV(MLPClassifier(), param_grid=hyperparam)
emotions_classifier_top_mlp_gridsearch_emb.fit(X=training_X_emotions_emb, y=training_dataset[:, 1])
emotions_classifier_prediction_tmlp_emb = emotions_classifier_top_mlp_gridsearch_emb.predict(X=testing_X_emotions_emb)
print(emotions_classifier_prediction_tmlp_emb)


In [ ]:
# Append Emotions results
performance_file = open("performance", "a")
performance_file.write(
    "-----Emotions classification (google model) (TOP Multi Layered Perception with word Embeddings)-----\n")

performance_file.write(
    f"Emotions hyperparameters = {emotions_classifier_top_mlp_gridsearch_emb.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 1], y_pred=emotions_classifier_prediction_tmlp_emb)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 1], y_pred=emotions_classifier_prediction_tmlp_emb, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")

performance_file.close()

In [ ]:
# For sentiments classifications
sentiments_classifier_top_mlp_gridsearch_emb = GridSearchCV(MLPClassifier(), param_grid=hyperparam)
sentiments_classifier_top_mlp_gridsearch_emb.fit(X=training_X_sentiments_emb, y=training_dataset[:, 2])
sentiments_classifier_prediction_tmlp_emb = sentiments_classifier_top_mlp_gridsearch_emb.predict(X=testing_X_emotions_emb)
print(sentiments_classifier_prediction_tmlp_emb)

In [ ]:
# Append Sentiments results
performance_file = open("performance", "a")
performance_file.write(
    "-----Sentiments classification (google model) (TOP Multi Layered Perception with word Embeddings)-----\n")

performance_file.write(
    f"Sentiments hyperparameters = {sentiments_classifier_top_mlp_gridsearch_emb.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 2], y_pred=sentiments_classifier_prediction_tmlp_emb)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 2], y_pred=sentiments_classifier_prediction_tmlp_emb, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")

performance_file.close()

3.7. □ (2pts) Display the performance of your classifiers using `metrics.classification_report` and add
these to your `performance` file.

3.8. □ (7.5pts) **Do your own exploration:** Rerun your best performing model but with 2 other English
pretrained embedding models and compare the results. Many pre-trained embeddings are available
on line (including in Gensim or at http://vectors.nlpl.eu/repository).

In [ ]:
fast_model = gensim.downloader.load("fasttext-wiki-news-subwords-300") 
twitter_model = gensim.downloader.load("glove-twitter-100")

[==================================================] 100.0% 958.5/958.4MB downloaded


Using the `fasttext-wiki-news-subwords-300` model from the gensim repo https://github.com/RaRe-Technologies/gensim-data

In [ ]:
# Repetition of 3.3 to 3.7 using the fast model
# Tokenization has been done in 3.2 so will reuse that code 
# phrase_token_vector = [nltk.tokenize.word_tokenize(i) for i in phrases]
# emotions_and_phrases_token_vector = [nltk.tokenize.word_tokenize(i) for i in emotions_and_phrases]


In [ ]:
mean_embedding_list_emotions = []
for i in phrase_token_vector:
    mean_embedding_list_emotions.append(fast_model.get_mean_vector(i))

mean_embedding_list_sentiments = []
for i in emotions_and_phrases_token_vector:
    mean_embedding_list_sentiments.append(fast_model.get_mean_vector(i))


In [ ]:
# Split the tokens (to get the hit rate)
training_phrase_token, testing_phrase_token = train_test_split(
    phrase_token_vector, train_size=0.8, test_size=0.2)

training_emotions_and_phrases_token, testing_emotions_and_phrases_token = train_test_split(
    emotions_and_phrases_token_vector, train_size=0.8, test_size=0.2)


# Split the mean embedding list for both eomtions and sentiments (the X values)
training_X_emotions_emb, testing_X_emotions_emb = train_test_split(
    mean_embedding_list_emotions, train_size=0.8, test_size=0.2)

training_X_sentiments_emb, testing_X_sentiments_emb = train_test_split(
    mean_embedding_list_sentiments, train_size=0.8, test_size=0.2)


In [ ]:
# Hit Rate for phrases training set
num_of_hits = 0
total = 0

for i in training_phrase_token:
    for word in i:
        try:
            # pdb.set_trace()
            fast_model.key_to_index[word]
            num_of_hits += 1
        except Exception:
            pass
        total += 1

print(f"Hit Rate in training set (phrases) = {num_of_hits/total*100}%")


# Hit Rate for phrases testing set
num_of_hits = 0
total = 0

for i in testing_phrase_token:
    for word in i:
        try:
            # pdb.set_trace()
            fast_model.key_to_index[word]
            num_of_hits += 1
        except Exception:
            pass
        total += 1

print(f"Hit Rate in testing set (phrases) = {num_of_hits/total*100}%")

# Hit Rate for phrases and emotions training set
num_of_hits = 0
total = 0

for i in training_emotions_and_phrases_token:
    for word in i:
        try:
            # pdb.set_trace()
            fast_model.key_to_index[word]
            num_of_hits += 1
        except Exception:
            pass
        total += 1

print(f"Hit Rate in training set (phrases and emotions) = {num_of_hits/total*100}%")


# Hit Rate for phrases and emotions testing set
num_of_hits = 0
total = 0

for i in testing_emotions_and_phrases_token:
    for word in i:
        try:
            # pdb.set_trace()
            fast_model.key_to_index[word]
            num_of_hits += 1
        except Exception:
            pass
        total += 1

print(f"Hit Rate in testing set (phrases and emotions) = {num_of_hits/total*100}%")


Hit Rate in training set (phrases) = 98.0262092439816%
Hit Rate in testing set (phrases) = 98.08410985869816%
Hit Rate in training set (phrases and emotions) = 98.15682028497747%
Hit Rate in testing set (phrases and emotions) = 98.13873316694064%


In [ ]:
# For emotions classification
emotions_classifier_mlp_embedding = MLPClassifier()

# Fit the model with training_X as X and columns of training_dataset as y
emotions_classifier_mlp_embedding.fit(X=training_X_emotions_emb,
                                      y=training_dataset[:, 1])

# Make predictions with testing_X as X
emotion_prediction_mlp_embedding = emotions_classifier_mlp_embedding.predict(
    X=testing_X_emotions_emb)
print(emotion_prediction_mlp_embedding)

d:\ProgramData\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


['neutral' 'neutral' 'neutral' ... 'neutral' 'neutral' 'neutral']


In [ ]:
# Append Emotions results
performance_file = open("performance", "a")
performance_file.write(
    "-----Emotions classification (fast model) (Multi Layered Perception with word Embeddings)-----\n")

performance_file.write(
    f"Emotions hyperparameters = {emotions_classifier_mlp_embedding.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 1], y_pred=emotion_prediction_mlp_embedding)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 1], y_pred=emotion_prediction_mlp_embedding, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")

performance_file.close()

In [ ]:
# For sentiments classification
sentiments_classifier_mlp_embedding = MLPClassifier()

# Fit the model with training_X as X and columns of training_dataset as y
sentiments_classifier_mlp_embedding.fit(X=training_X_sentiments_emb,
                                      y=training_dataset[:, 2])

# Make predictions with testing_X as X
sentiments_prediction_mlp_embedding = sentiments_classifier_mlp_embedding.predict(
    X=testing_X_sentiments_emb)
print(sentiments_prediction_mlp_embedding)

d:\ProgramData\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


['positive' 'positive' 'positive' ... 'neutral' 'neutral' 'neutral']


In [ ]:
# Append Sentiments results
performance_file = open("performance", "a")
performance_file.write(
    "-----Sentiments classification (concept model) (Multi Layered Perception with word Embeddings)-----\n")

performance_file.write(
    f"Sentiments hyperparameters = {sentiments_classifier_mlp_embedding.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 2], y_pred=sentiments_prediction_mlp_embedding)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 2], y_pred=sentiments_prediction_mlp_embedding, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")

performance_file.close()


Using the Twitter (`glove-twitter-100`) Model from the gensim repo https://github.com/RaRe-Technologies/gensim-data

In [ ]:
# Repetition of 3.3 to 3.7 using the twitter model
# Tokenization has been done in 3.2 so will reuse that code 
# phrase_token_vector = [nltk.tokenize.word_tokenize(i) for i in phrases]
# emotions_and_phrases_token_vector = [nltk.tokenize.word_tokenize(i) for i in emotions_and_phrases]


In [ ]:
mean_embedding_list_emotions = []
for i in phrase_token_vector:
    mean_embedding_list_emotions.append(twitter_model.get_mean_vector(i))

mean_embedding_list_sentiments = []
for i in emotions_and_phrases_token_vector:
    mean_embedding_list_sentiments.append(twitter_model.get_mean_vector(i))


In [ ]:
# Split the tokens (to get the hit rate)
training_phrase_token, testing_phrase_token = train_test_split(
    phrase_token_vector, train_size=0.8, test_size=0.2)

training_emotions_and_phrases_token, testing_emotions_and_phrases_token = train_test_split(
    emotions_and_phrases_token_vector, train_size=0.8, test_size=0.2)


# Split the mean embedding list for both eomtions and sentiments (the X values)
training_X_emotions_emb, testing_X_emotions_emb = train_test_split(
    mean_embedding_list_emotions, train_size=0.8, test_size=0.2)

training_X_sentiments_emb, testing_X_sentiments_emb = train_test_split(
    mean_embedding_list_sentiments, train_size=0.8, test_size=0.2)


In [ ]:
# Hit Rate for phrases training set
num_of_hits = 0
total = 0

for i in training_phrase_token:
    for word in i:
        try:
            # pdb.set_trace()
            twitter_model.key_to_index[word]
            num_of_hits += 1
        except Exception:
            pass
        total += 1

print(f"Hit Rate in training set (phrases) = {num_of_hits/total*100}%")


# Hit Rate for phrases testing set
num_of_hits = 0
total = 0

for i in testing_phrase_token:
    for word in i:
        try:
            # pdb.set_trace()
            twitter_model.key_to_index[word]
            num_of_hits += 1
        except Exception:
            pass
        total += 1

print(f"Hit Rate in testing set (phrases) = {num_of_hits/total*100}%")

# Hit Rate for phrases and emotions training set
num_of_hits = 0
total = 0

for i in training_emotions_and_phrases_token:
    for word in i:
        try:
            # pdb.set_trace()
            twitter_model.key_to_index[word]
            num_of_hits += 1
        except Exception:
            pass
        total += 1

print(f"Hit Rate in training set (phrases and emotions) = {num_of_hits/total*100}%")


# Hit Rate for phrases and emotions testing set
num_of_hits = 0
total = 0

for i in testing_emotions_and_phrases_token:
    for word in i:
        try:
            # pdb.set_trace()
            twitter_model.key_to_index[word]
            num_of_hits += 1
        except Exception:
            pass
        total += 1

print(f"Hit Rate in testing set (phrases and emotions) = {num_of_hits/total*100}%")


Hit Rate in training set (phrases) = 84.53000788423333%
Hit Rate in testing set (phrases) = 84.57877327284756%
Hit Rate in training set (phrases and emotions) = 85.48617666531642%
Hit Rate in testing set (phrases and emotions) = 85.43728210336387%


In [ ]:
# For emotions classification
emotions_classifier_mlp_embedding = MLPClassifier()

# Fit the model with training_X as X and columns of training_dataset as y
emotions_classifier_mlp_embedding.fit(X=training_X_emotions_emb,
                                      y=training_dataset[:, 1])

# Make predictions with testing_X as X
emotion_prediction_mlp_embedding = emotions_classifier_mlp_embedding.predict(
    X=testing_X_emotions_emb)
print(emotion_prediction_mlp_embedding)

['neutral' 'neutral' 'neutral' ... 'neutral' 'neutral' 'neutral']


d:\ProgramData\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
# Append Emotions results
performance_file = open("performance", "a")
performance_file.write(
    "-----Emotions classification (twitter model) (Multi Layered Perception with word Embeddings)-----\n")

performance_file.write(
    f"Emotions hyperparameters = {emotions_classifier_mlp_embedding.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 1], y_pred=emotion_prediction_mlp_embedding)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 1], y_pred=emotion_prediction_mlp_embedding, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")

performance_file.close()

In [ ]:
# For sentiments classification
sentiments_classifier_mlp_embedding = MLPClassifier()

# Fit the model with training_X as X and columns of training_dataset as y
sentiments_classifier_mlp_embedding.fit(X=training_X_sentiments_emb,
                                      y=training_dataset[:, 2])

# Make predictions with testing_X as X
sentiments_prediction_mlp_embedding = sentiments_classifier_mlp_embedding.predict(
    X=testing_X_sentiments_emb)
print(sentiments_prediction_mlp_embedding)

d:\ProgramData\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


['negative' 'positive' 'positive' ... 'neutral' 'positive' 'ambiguous']


In [ ]:
# Append Sentiments results
performance_file = open("performance", "a")
performance_file.write(
    "-----Sentiments classification (twitter model) (Multi Layered Perception with word Embeddings)-----\n")

performance_file.write(
    f"Sentiments hyperparameters = {sentiments_classifier_mlp_embedding.n_features_in_}\n")

confusion_matrix_output = confusion_matrix(
    y_true=testing_dataset[:, 2], y_pred=sentiments_prediction_mlp_embedding)
performance_file.write(f"Confusion Matrix = \n{confusion_matrix_output}\n\n")

class_report = classification_report(
    y_true=testing_dataset[:, 2], y_pred=sentiments_prediction_mlp_embedding, zero_division=0)
performance_file.write(f"Classification Report = \n{class_report}\n")
performance_file.write(
    f"----------------------------------------------------------\n\n")

performance_file.close()
